In [5]:
# Imports necesary libraries
import requests
import time
import json
import numpy as np

# Config for API requests
api_key = 'YOUR API KEY'
server = 'NA1' # NA1, EUW1, AP-SOUTHEAST-1, etc.
region = 'americas' # Will be different based on your server
queue = 'RANKED_SOLO_5x5' # Game type for ranked match
queue_id = 420 # Solo ranked matches only
players = 1000 # Number of players from a ranked queue
num_games = 10 # Number of games played per player

In [6]:
summoners_url = f'https://{server}.api.riotgames.com/lol/league/v4/masterleagues/by-queue/{queue}?api_key={api_key}'
response = requests.get(summoners_url)
summoners = response.json()

In [7]:
summoner_names = []
for i in range(players): # Chooses the players from the Master's league
    name = summoners['entries'][i]['summonerName'] # Process the result and extract summoner name
    summoner_names.append(name)

In [8]:
def get_player_ids():
    puuid_list = []
    global summoner_names
    counter = players
    
    while counter > 0:
        for _ in range(5): # Request in a batch of five to meet the rate limit of 100 requests every 2 minutes
            limit = min(20, len(summoner_names))      
            time.sleep(1.5)
            for i in range(limit): # 20 requests per second
                summoner = summoner_names[i]
                puuid_url = f'https://{server}.api.riotgames.com/lol/summoner/v4/summoners/by-name/{summoner}?api_key={api_key}'
                response = requests.get(puuid_url)
                if response.status_code == 429:
                    print("Exceeded API rate limit")
                    time.sleep(120)
                    continue
                player = response.json()
                try: # Unsure why but some players do not have puuid
                    puuid_list.append(player["puuid"]) # Process the result and extract player IDs
                except:
                    continue
            counter -= limit
            summoner_names = summoner_names[limit:]
        time.sleep(120) # 100 requests every 2 minutes
        
    return puuid_list

In [9]:
def get_match_ids():
    match_ids = []
    global puuid_list
    counter = len(puuid_list)
    
    while counter > 0:
        for _ in range(5): # Makes five of 20 requests per second
            limit = min(20, len(puuid_list))
            time.sleep(1.5)
            for i in range(limit): # 20 requests per second
                puuid = puuid_list[i]
                match_id_url = f'https://{region}.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?queue={queue_id}&start=0&count={num_games}&api_key={api_key}'
                response = requests.get(match_id_url)
                if response.status_code == 429:
                    print("Exceeded API rate limit")
                    time.sleep(120)
                    continue
                match_id = response.json()
                match_ids.extend(match_id) # Process the result and extract match IDs
            counter -= limit
            puuid_list = puuid_list[limit:]
        time.sleep(120) # 100 requests every 2 minutes
        
    return match_ids

In [10]:
def get_match_data():
    match_list = []
    global match_ids
    unique_ids = list(set(match_ids)) # Remove any duplicate matches by chance
    counter = len(unique_ids)
    
    while counter > 0:
        for _ in range(5): # Makes five of 20 requests per second
            limit = min(20, len(unique_ids))
            time.sleep(1.5)
            for i in range(limit): # 20 requests per second
                match_id = unique_ids[i]
                match_url = f'https://{region}.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}'
                response = requests.get(match_url)
                if response.status_code == 429:
                    print("Exceeded API rate limit")
                    time.sleep(120)
                    continue
                match = response.json()
                match_list.append(match) # Process the result and extract match
            counter -= limit
            unique_ids = unique_ids[limit:]
        time.sleep(120) # 100 requests every 2 minutes
        
    return match_list

In [11]:
time.sleep(120)
puuid_list = get_player_ids()

In [12]:
time.sleep(120)
match_ids = get_match_ids()

In [13]:
time.sleep(120)
matches = get_match_data()

In [ ]:
with open("data/matches.jsonl", "w") as file:
    for match in matches:
        json.dump(match, file)
        file.write("\n")